<a href="https://colab.research.google.com/github/Elwing-Chou/ml0804/blob/master/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets.mnist import load_data
# ((訓練圖片, 訓練答案), (驗證圖片, 驗證答案))
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
import matplotlib.pyplot as plt
import random
idx = random.randint(0, 59999)
print(y_train[idx])
# cmap="gray"
plt.imshow(x_train[idx], cmap="Reds_r")

In [ ]:
import pandas as pd
pd.DataFrame(x_train[idx])

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
layers = [
    # 784 * 128 + 128(bias)
    Dense(256, activation="relu", input_dim=784),
    # 128 * 10(連線個數) + 10(bias)
    Dense(10, activation="softmax")
]
model = Sequential(layers)
model.summary()

In [ ]:
# 交叉商: p1 * log(1/q1) + p2 * log(1/q2)....
# 多個: Categorical CrossEntropy
# 一個: Binary CrossEntropy p * log(1/q) + (1-p) * log(1/1-q)
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(loss=CategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
# 資料處理: y
from tensorflow.keras.utils import to_categorical
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)
print(y_train[0])
print(y_train_cat[0])

In [ ]:
# 資料處理: x
# reshape: -1(numpy會幫你算該等於多少, 只能一個)
x_train_norm = x_train.reshape(-1, 784) / 255
x_test_norm = x_test.reshape(-1, 784) / 255
print(x_train_norm.shape)
print(x_test_norm.shape)

In [ ]:
# batch:看幾筆在調整一次(20-200)
# epochs:整份訓練資料看幾遍 (10->60000*10)
# verbose: 0(quiet) 1(default) 2(我喜歡的)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callbacks = [
    ModelCheckpoint("mlp.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_norm,
     y_train_cat,
     validation_split=0.1,
     batch_size=200,
     epochs=100,
     verbose=2,
     callbacks=callbacks)

In [ ]:
model.evaluate(x_test_norm, y_test_cat)

In [ ]:
from sklearn.metrics import confusion_matrix
pre = model.predict_classes(x_test_norm)
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat, 
       columns=["{}(預測)".format(i) for i in range(10)],
       index=["{}(真實)".format(i) for i in range(10)])

In [ ]:
import numpy as np
idx = np.nonzero(pre != y_test)[0]
img, label, pred = x_test[idx], y_test[idx], pre[idx]

plt.figure(figsize=(14, 42))
width = 10
height = len(img) // width + 1
for i in range(len(img)):
    plt.subplot(height, width, i+1)
    t = "[T]:{}\n[P]:{}".format(label[i], pred[i])
    plt.title(t)
    plt.axis("off")
    plt.imshow(img[i], cmap="Reds_r")

In [ ]:
# https://pillow.readthedocs.io/en/5.1.x/handbook/concepts.html#concept-modes
from tensorflow.keras.preprocessing.image import load_img
fn = input("檔名:")
img = load_img(fn, target_size=(28, 28)).convert("L")
img_norm = np.array(img).reshape(1, 784) / 255
ans = model.predict_classes(img_norm)
print("答案:", ans[0])
prob = model.predict(img_norm)[0]
for i, p in enumerate(prob):
    print(i, "的機率:", round(p, 3))
plt.imshow(img, cmap="gray")